In [ ]:
import numpy as np
import tensorflow as tf
from PIL import Image
import sys

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path="student_model_tiny.tflite")
interpreter.allocate_tensors()

# Get input and output tensors info
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Function to preprocess the input image
def preprocess_image(image_path):
    image = Image.open(image_path).convert('RGB')
    image = image.resize((64, 64))
    image = np.asarray(image).astype(np.float32) / 255.0

    # Normalize using the same mean and std used during training
    mean = np.array([0.485, 0.456, 0.406], dtype=np.float32)
    std = np.array([0.229, 0.224, 0.225], dtype=np.float32)
    image = (image - mean) / std

    # Convert HWC -> CHW
    image = image.transpose(2, 0, 1)

    # Add batch dimension
    image = np.expand_dims(image, axis=0).astype(np.float32)
    return image


for i in range(12):
    # Load and preprocess input imag
    image_path = f"./images/image_{i}.png"
    input_data = preprocess_image(image_path)

    # Ensure input type matches the model's expected type
    input_index = input_details[0]['index']
    interpreter.set_tensor(input_index, input_data)

    # Run inference
    interpreter.invoke()

    # Get the output
    output_index = output_details[0]['index']
    output_data = interpreter.get_tensor(output_index)

    # Get predicted class
    predicted_class = np.argmax(output_data)
    print(f"Predicted class: {predicted_class}, Scores: {output_data}")


: 

In [3]:
import numpy as np
import tensorflow as tf
from PIL import Image
import os
import sys

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path="student_model_tiny.tflite")
interpreter.allocate_tensors()

# Get input and output tensors info
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Print model input details for debugging
print(f"Input details: {input_details}")
print(f"Expected input shape: {input_details[0]['shape']}")
print(f"Expected input type: {input_details[0]['dtype']}")

# Function to preprocess the input image
def preprocess_image(image_path):
    if not os.path.exists(image_path):
        print(f"Error: Image {image_path} not found")
        return None
        
    image = Image.open(image_path).convert('RGB')
    image = image.resize((64, 64))
    image = np.asarray(image).astype(np.float32) / 255.0

    # Normalize using the same mean and std used during training
    mean = np.array([0.485, 0.456, 0.406], dtype=np.float32)
    std = np.array([0.229, 0.224, 0.225], dtype=np.float32)
    
    # Apply normalization correctly
    image = (image - mean) / std

    # Convert HWC -> CHW (PyTorch format)
    image = image.transpose(2, 0, 1)
    
    # Add batch dimension
    image = np.expand_dims(image, axis=0).astype(np.float32)
    
    # Check if we need to convert back to HWC for TFLite
    if input_details[0]['shape'][3] == 3:  # If the last dimension is 3 (channels)
        # Convert back to HWC for TFLite
        image = image.transpose(0, 2, 3, 1)
        
    return image

for i in range(12):
    # Load and preprocess input image
    image_path = f"./images/image_{i}.png"
    input_data = preprocess_image(image_path)
    
    if input_data is None:
        continue

    # Ensure input type matches the model's expected type
    input_index = input_details[0]['index']
    
    # Check if we need to quantize the input
    if input_details[0]['dtype'] != np.float32:
        print(f"Converting input to {input_details[0]['dtype']}")
        input_data = input_data.astype(input_details[0]['dtype'])
    
    interpreter.set_tensor(input_index, input_data)

    # Run inference
    interpreter.invoke()

    # Get the output
    output_index = output_details[0]['index']
    output_data = interpreter.get_tensor(output_index)

    # Get predicted class
    predicted_class = np.argmax(output_data)
    confidence = output_data[0][predicted_class]
    print(f"Image {i}: Predicted class: {predicted_class}, Confidence: {confidence:.4f}  Full scores: {output_data[0]}")
    # print(f"Full scores: {output_data[0]}")


Input details: [{'name': 'serving_default_input:0', 'index': 0, 'shape': array([ 1,  3, 64, 64], dtype=int32), 'shape_signature': array([ 1,  3, 64, 64], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
Expected input shape: [ 1  3 64 64]
Expected input type: <class 'numpy.float32'>
Image 0: Predicted class: 0, Confidence: -6.0072  Full scores: [-6.0072184 -7.4837065]
Image 1: Predicted class: 0, Confidence: -8.3040  Full scores: [ -8.303981 -10.891712]
Image 2: Predicted class: 1, Confidence: -4.0905  Full scores: [-6.4334745 -4.0905094]
Image 3: Predicted class: 0, Confidence: -6.0769  Full scores: [-6.0769405 -7.6767435]
Image 4: Predicted class: 1, Confidence: 0.4733  Full scores: [-9.711008    0.47329766]
Image 5: Predicted class: 1, Confidence: -3.4028  Full scores: [-8.992496 -3.402752]
Image 6: Pre

In [18]:
import numpy as np
import tensorflow as tf
from PIL import Image
import sys

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path="student_model_tiny.tflite")
interpreter.allocate_tensors()

# Get input and output tensors info
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Function to preprocess the input image
def preprocess_image(image_path):
    image = Image.open(image_path).convert('RGB')
    image = image.resize((64, 64))
    image = np.asarray(image).astype(np.float32) / 255.0

    # Normalize using the same mean and std used during training
    mean = np.array([0.485, 0.456, 0.406], dtype=np.float32)
    std = np.array([0.229, 0.224, 0.225], dtype=np.float32)
    image = (image - mean) / std

    # Convert HWC -> CHW
    image = image.transpose(2, 0, 1)

    # Add batch dimension
    image = np.expand_dims(image, axis=0).astype(np.float32)
    return image


# Load and preprocess input imag
image_path = f"./fake/id2_id4_0007_frame_23.png"
input_data = preprocess_image(image_path)

# Ensure input type matches the model's expected type
input_index = input_details[0]['index']
interpreter.set_tensor(input_index, input_data)

# Run inference
interpreter.invoke()

# Get the output
output_index = output_details[0]['index']
output_data = interpreter.get_tensor(output_index)

# Get predicted class
predicted_class = np.argmax(output_data)
print(f"Predicted class: {predicted_class}, Scores: {output_data}")


Predicted class: 1, Scores: [[-6.688732 -5.56662 ]]
